In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import DeepFM
from deepctr.inputs import  SparseFeat, DenseFeat,get_feature_names
from sklearn.metrics import mean_squared_error

In [51]:
data=pd.read_csv('./ratings.csv')

In [52]:
print(data.head)
print(data.shape)

<bound method NDFrame.head of          userId  movieId  rating   timestamp
0             1        2     3.5  1112486027
1             1       29     3.5  1112484676
2             1       32     3.5  1112484819
3             1       47     3.5  1112484727
4             1       50     3.5  1112484580
...         ...      ...     ...         ...
1048570    7120      168     5.0  1175543061
1048571    7120      253     4.0  1175542225
1048572    7120      260     5.0  1175542035
1048573    7120      261     4.0  1175543376
1048574    7120      266     3.5  1175542454

[1048575 rows x 4 columns]>
(1048575, 4)


In [53]:
sparse_features = ["userId", "movieId"]
target = ['rating']
data[sparse_features] = data[sparse_features].fillna('-1', )
sparse_feature_columns = [SparseFeat(feat, data[feat].nunique())
                        for feat in sparse_features]


In [54]:
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

In [55]:
dnn_feature_columns = sparse_feature_columns 
linear_feature_columns = sparse_feature_columns 

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
print(feature_names)

['userId', 'movieId']


In [56]:
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}


使用DeepFM进行训练



In [57]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values,batch_size=256, epochs=1, verbose=True, validation_split=0.2,)


Train on 671088 samples, validate on 167772 samples
671088/671088 [==============================] - 40s 60us/sample - loss: 0.9306 - mean_squared_error: 0.9293 - val_loss: 0.7432 - val_mean_squared_error: 0.7407


 使用DeepFM进行预测

In [58]:
pred_ans = model.predict(test_model_input, batch_size=256)

mse=round(mean_squared_error(test[target].values,pred_ans),4)
rmse=mse**0.5
print('test RMSE',rmse)


test RMSE 0.8609878047916822
